### In this notebook I will try to repair hyperex amplicon extraction tool

hyperex keeps crushing when there is forward primer found upstream revese primer:

thread 'main' panicked at /home/mg/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/hyperex-0.1.1/src/utils.rs:420:45: slice index starts at 256 but ends at 65 note: run with \`RUST_BACKTRACE=1` environment variable to display a backtrace Aborted (core dumped)   

I will  load the fasta seq with Biopython and then use hyperex on each read

In [8]:
import pandas as pd
import os
print(os.getcwd())

In [9]:
from Bio import SeqIO
fasta_seq = list(SeqIO.parse("datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/untrimmed_sequences/uchime_reference_dataset_untrimmed_16_10_2022.fasta", "fasta"))
#fasta_seq = fasta_seq[:300]
print((fasta_seq[:5]))


In [10]:
import pandas as pd
import re
primers_df = pd.read_excel("my_data/primers.xlsx",header=1)
primers_df["better_names"] = primers_df["homogenized primer names"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\-\.\_]', '_', str(x).replace("/","___")))
primers_df_filtered = primers_df[~primers_df["primers sequences"].str.contains("[\\(a-z\\+]")]
primers_df_filtered


In [11]:
from Bio.Seq import Seq
Seq("GCTGCGTTCTTCATCGATGC").reverse_complement()

In [12]:
from scripts.hyperex_repaired import hyperex_repaired

## TEST hyperex_repaired function

In [13]:
#test hyperex_repaired on test_hyperex_repaired.fa
test_fasta_seq = list(SeqIO.parse("scripts/test_hyperex_repaired.fa", "fasta"))
test_res_seq = hyperex_repaired(test_fasta_seq,"TCCGTAGGTGAACCTGCGG","GCTGCGTTCTTCATCGATGC")
print("All  reads:\n",", ".join([i.id for i in test_fasta_seq]),sep="")
print("Good reads:\n",", ".join([i.id for i in test_res_seq]),sep="")
print("Bad reads:\n",", ".join(set([i.id for i in test_fasta_seq])-set([i.id for i in test_res_seq])),sep="")
test_res_seq

## Extract amplicons with hyperex repaired

In [14]:
from Bio import SeqIO
import sys

lenky = {}
seqs_list = []
for n,row in primers_df_filtered.iterrows():
    _,primer_seq,region,_,better_name, = list(row)
    forw = primer_seq.split("/")[0].strip()
    rev = primer_seq.split("/")[1].strip()
    print(better_name)
    res_seq = hyperex_repaired(fasta_seq,forw,rev)
    output_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{better_name}.fasta"
    with open(output_file, "w") as handle:
        SeqIO.write(res_seq, handle, "fasta-2line")
    print(n)
    lenky[better_name] = len(res_seq)
    seqs_list.append(res_seq)


In [ ]:
ids = {}
for p,seq in zip(primers_df_filtered["better_names"],seqs_list):
    ids[p] = [i.id for i in seq]

In [ ]:
from itertools import chain
ids_one_list = chain.from_iterable([v for k, v in ids.items()])
from collections import Counter
counts = Counter(ids_one_list)
len(counts),len(fasta_seq)

## Cut primers with cutadapt - this is useless - I should do it at the end

In [ ]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import os
from subprocess import run

# Function to sanitize sequence IDs and descriptions
def sanitize_fasta_records(records):
    sanitized_records = []
    for record in records:
        sanitized_id = ''.join(c if c.isascii() or c in "-_." else "_" for c in record.id)
        sanitized_description = ''.join(c if c.isascii() or c in "-_." else "_" for c in record.description)
        sanitized_records.append(SeqRecord(Seq(str(record.seq)), id=sanitized_id, description=sanitized_description))
    return sanitized_records

# Iterate over each primer and its corresponding file
for _, row in primers_df_filtered.iterrows():
    primer_name = row["better_names"]
    primer_seq = row["primers sequences"]
    input_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}.fasta"
    sanitized_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}_sanitized.fasta"
    output_file = f"../datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}_cut.fasta"
    
    # Sanitize the input FASTA file
    records = list(SeqIO.parse(input_file, "fasta"))
    sanitized_records = sanitize_fasta_records(records)
    with open(sanitized_file, "w") as handle:
        SeqIO.write(sanitized_records, handle, "fasta")
    
    # Run cutadapt to cut primers
    command = [
        "cutadapt",
        "-g", primer_seq.split("/")[0].strip(),  # Forward primer
        "-a", primer_seq.split("/")[1].strip(),  # Reverse primer
        "-o", output_file,
        sanitized_file
    ]
    result = run(command, capture_output=True, text=True)
    
    # Print the result for debugging
    print(f"Processed {primer_name}:")
    print(result.stdout)
    print(result.stderr)

### use Simera on fastas from hyperex

In [15]:
skipped = []

In [ ]:
import os
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from time import sleep
from subprocess import run
import time
from subprocess import TimeoutExpired

# Function to append "_1" to each read name in the FASTA file
def append_suffix_to_fasta(input_file, output_file, suffix="_1"):
    records = list(SeqIO.parse(input_file, "fasta"))
    modified_records = []
    for record in records:
        record.id = f"{record.id}{suffix}"
        record.description = ""  # Clear description to avoid duplication
        modified_records.append(record)
    with open(output_file, "w") as handle:
        SeqIO.write(modified_records, handle, "fasta-2line")

# Iterate over each primer and its corresponding file
for numero,row in enumerate(primers_df_filtered.iterrows()):
    if numero < 0: continue
    
    row = row[1]
    primer_name = row["better_names"]
    primer_seq = row["primers sequences"]
    forward_primer = primer_seq.split("/")[0].strip()
    reverse_primer = primer_seq.split("/")[1].strip()
        # Check if the FASTA file is empty

    if primer_name != "gITS7___ITS4": print("skipping....") ; continue
    print(primer_name)
    input_file = f"datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}_sanitized.fasta"
    modified_file = f"datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_hyperex/{primer_name}_abund.fasta"
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    output_dir = f"datasets/uchime_reference_dataset_16_10_2022/2022_10_26_chimera_reference_release/amplicons_from_primers_simera/{primer_name}/{timestamp}/"
    records = list(SeqIO.parse(input_file, "fasta"))
    if len(records) < 10:  # If no records are found, continue to the next iteration
        print(f"Skipping {primer_name} as the file is empty.")
        continue
    # Append "_1" to each read name
    append_suffix_to_fasta(input_file,modified_file)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    print("simera!!!",primer_name,forward_primer,reverse_primer)
    # Run Simera
    command = [
        "Simera/Simera",
        "-i", modified_file,
        "-o", output_dir,
        "-n", "25",
        "-s", "10000",
        "-l", "0.00005",
        "-c", "100000", # too large values for big fastas (6+Mb) will freeze Simera sometimes ??? dunno why, it fries my pc tho
        "-f", forward_primer,                      
        "-r", reverse_primer,
        "-x"
    ] # Simera is stuck when there is too little seqs in the fasta
    #result = run(command, capture_output=True, text=True)
    try:
        result = run(command, capture_output=True, text=True, timeout=240)  # Timeout after 300 seconds
    except TimeoutExpired:
        print(f"Simera timed out for {primer_name}. Skipping...")
        skipped.append((numero,primer_name))
        continue  # Skip to the next iteration
 
    # Print the result for debugging
    print(f"Processed {primer_name}:")
    #print(result.stdout)
    #print(result.stderr)
    print(numero+1,"/",len(primers_df_filtered))
    sleep(0)

In [ ]:
from Bio.Seq import Seq
Seq("GTTCAAAGAYTCGATGATTCAC").reverse_complement()
from random import choice
ss = "".join([choice("ACGT") for i in range(370)])
ss
#len("GTAAAAGTCGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTATTGAAATAAACTTAGGGGTTGTTGCTGGCCTTTCAAGGGCAATTTTGTGCACGCCTCTATCTATTTCCCTTTAACACCCCCATTGTGCATCTTGTGTAGGTTAACGTAAAGTTAATCTATGTTTTTATTTTTATACCTTTGTTTTTGAATGGGAATGTAATTTTTTAAACCCTCAGTAAGCTCTCATTTTGGAGCTGAAAAGGAAATAAATATGTGTACAACTTTCAACAACGGATCTCTTGGCTCTCGCATCGATGAAGAACGCAGCGAAATGCGATAAGTAATGTGAATTGCAGAATTCAGTGAATCATCGAATCTTTGAAC")


### END

## OLD versions:

In [ ]:
# old version
import os
import subprocess
import re
from collections import Counter
import sys

results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = f"hyperex_temp_out.fa"
hyperex_log = f"hyperex.log"
hyperex_gff = f"hyperex_temp_out.gff"

# Clean up any existing temporary files
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)

# Command template for hyperex
command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out"]

lenka = len(fasta_seq)
for i, record in enumerate(fasta_seq):
    if i % 50 == 0: print(f"{i}/{lenka}", end="\r")
    if i >0: command = ["hyperex", "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC", "-p", "hyperex_temp_out","--force"] 
    # Convert the record to FASTA format string
    fasta_data = f">{record.id}\n{str(record.seq)}\n"
    
    # Run hyperex with the FASTA data passed via stdin
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=fasta_data)
    
    # Collect warnings and errors
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]", ""), re.findall("WARN(.+)", stdout))))
    errors.extend(stderr)
    
    # Read results if the output file exists
    if os.path.exists(hyperex_out_fa):
        with open(hyperex_out_fa, "r") as fa_file:
            results.append(fa_file.read())

# Clean up any remaining temporary files
if os.path.exists(hyperex_out_fa): os.remove(hyperex_out_fa)
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)

print()
real_results = [i for i in results if len(i) > 0]
print("number of real results:", len(real_results))
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("ERRORS:", error_counts, "WARNINGS:", warning_counts, sep="\n")


In [ ]:
# old version 2
import os
import subprocess
#from time import sleep
#import sys
import re
from collections import Counter
results = []  # List to store the content of hyperex_out.fa files
warnings = []  # List to store the warnings
errors = []  # List to store the errors
hyperex_out_fa = f"hyperex_temp_out.fa"
hyperex_log = f"hyperex.log"
hyperex_gff = f"hyperex_temp_out.gff"
temp_filename = f"hyperex_temp_res.fasta"
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(temp_filename): os.remove(temp_filename)  # Delete the temporary input file

with open(temp_filename, "w") as temp_fasta:
    SeqIO.write(fasta_seq[0], temp_fasta, "fasta")
command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename]
result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
warnings.extend(list(map(lambda x: x.replace("\x1b[0m]",""), re.findall("WARN(.+)",result.stdout))))
errors.extend(result.stderr)
if os.path.exists(hyperex_out_fa):
    with open(hyperex_out_fa, "r") as fa_file:
        results.append(fa_file.read())
#print(result.stdout)
#print(result.stderr)
lenka = len(fasta_seq)
command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename,"--force"]
for i, record in enumerate(fasta_seq[1:]):
    #temp_filename = f"hyperex_temp_res.fasta"
    if i % 50 == 0: print(f"{i}/{lenka}",end="\r")
    with open(temp_filename, "w") as temp_fasta:
        SeqIO.write(record, temp_fasta, "fasta")
    #if i == 0:command = ["hyperex",  "-f", "GTGARTCATCGARTCTTTG", "-r", "TCCTCCGCTTATTGATATGC","-p","hyperex_temp_out", temp_filename]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    warnings.extend(list(map(lambda x: x.replace("\x1b[0m]",""), re.findall("WARN(.+)",result.stdout))))
    errors.extend(result.stderr)
    #if os.path.exists(hyperex_out_fa):
    with open(hyperex_out_fa, "r") as fa_file:
        results.append(fa_file.read())

if os.path.exists(hyperex_out_fa):os.remove(hyperex_out_fa)  
if os.path.exists(hyperex_log): os.remove(hyperex_log)
if os.path.exists(hyperex_gff): os.remove(hyperex_gff)
if os.path.exists(temp_filename): os.remove(temp_filename)  # Delete the temporary input file

print()
real_results = [i for i in results if len(i) >0]
print("number of real results:",len(real_results))
warning_counts = Counter(warnings)
error_counts = Counter(errors)
print("ERRORS:",list(error_counts),"WARNINGS:","\n".join(list(warning_counts)),sep="\n")

